<img src='../images/bildungscampus_logo.png' width="40%" align="left" />
<img src='../images/hhn.png' width="25%" align="right" />

# Schritt 4: Action Execution
Masterarbeit - Sebastian Kahlert | Fakultät Wirtschaft und Verkehr | Wirtschaftsinformatik - Informationsmanagement und Data Science | WS 2021/22

<img src='../images/bar.png'/>

## Generierung von SQL-Queries zur Extraktion und Speicherung neuer Informationen

Hier wird eine neue Funktion implementiert, die den ermittelten Intent und die extrahierten Informationen aus dem Slot Filling als Input nimmt und SQL Queries generiert, um neue Professoren aus dem Eingaben des Nutzers in die Datenbank aufzunehmen. Nachdem der Chatbot erkannt hat, dass der Benutzer einen neuen Professor mitgeben möchte, frägt dieser den Benutezr nach verschiedenen Informationen um die Datenbank füllen zu können. 

### 4.1. Import benötigter Bibliotheken

In [3]:
# import of necessary libraries
import nltk 
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

import datefinder
import numpy
import tflearn 
import tensorflow
import random 
import json
import pickle
import spacy
import sqlite3
import re
import os
import sqlite3
from difflib import SequenceMatcher

Instructions for updating:
non-resource variables are not supported in the long term
curses is not supported on this machine (please install/reinstall curses for an optimal experience)
Scipy not supported!


### 4.2. Intent Classification

Das Neuronale Netzwerk wurde zuvor in einem separaten Notebook erstellt, trainert und gespeichert. Der folgende Code versucht das gespeciherte Neuronale Netzwerk zu laden. Falls dies nicht möglich sein sollte wird ein neues Neuronales Netzwerk erstellt und trainiert. Hierzu werden die Trainingsdaten "data.pickle" geladen und die Layer des Netztes definiert.

In [4]:
# open training data for neural net
with open("../2. Intent Classififcation/data.pickle", "rb") as f:
    words, labels, training, output = pickle.load(f)

In [5]:
# open saved model or implement new one if model does not exist yet
tensorflow.compat.v1.reset_default_graph()
    
# Creating the Neural Network
net = tflearn.input_data(shape= [None, len(training[0])]) #input layer Neurons = numer of words in training
net = tflearn.fully_connected(net, 8) #hidden layer fully connected with 8 neuron
net = tflearn.fully_connected(net, len(output[0]), activation="softmax" ) #output layer 6 Neurons = labels
net = tflearn.regression(net)

model = tflearn.DNN(net)

try:
    model.load(r"..\2. Intent Classififcation\model.tflearn")
except:
    model.fit(training, output, n_epoch=1000, batch_size=8, show_metric=True)
    model.save(r"..\2. Intent Classififcation\model.tflearn")

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
INFO:tensorflow:Restoring parameters from C:\Users\Sebi\OneDrive\Studium\Thesis_Chatbot\2. Intent Classififcation\model.tflearn


In [6]:
model

In [7]:
# transform input in numbers to make it readable for neural net
def bag_of_words(s, words):
    bag = [0 for _ in range(len(words))]
    
    s_words = nltk.word_tokenize(s)
    s_words = [stemmer.stem(word.lower()) for word in s_words]
    
    for se in s_words:
        for i, w in enumerate(words):
            if w == se:
                bag[i] = 1
    
    return numpy.array(bag)

###  4.4. Action Execution 

Je nach intent wird ein vordefiniertes SQL-Query mit Platzhalter ausgeführt. Zuvor müssen die Platzhalter mit den extrahierten Informationen aus dem Slot Filling befüllt werden. Output ist die Information aus der Datenbank

In [14]:
# load NER model to identify names of new professor
nlp_name = spacy.load("en_core_web_md")

In [20]:
def action_execution_new_prof():
    
    # get name of new professor
    print("Great! What is the name of the new Professor?")
    inp = input("You: ")
    doc = nlp_name(inp)
    for ent in doc.ents:
        if ent.label_ == "PERSON":
            first_name = ent.text.split()[0]
            last_name = ent.text.split()[1]
        else:
            first_name = "Couldn't extract the name"
            last_name ="Couldn't extract the name"
    
    # get phone number
    print("What is the phone number")
    inp = input("You: ")
    
    re_number_1 = r"\D[\d]{2}? [\d]{4} [\d]{3} [\d]{4}"
    re_number_2 = r"\D[\d]{2}? [\d]{4} [\d]{3} [\d]{3}"
    re_number_3 = r"\D[\d]{2} [\(][\d]{1}[\)] [\d]{4} [\d]{3} [\d]{4}"
    
    try:
        telephone = re.compile("(%s|%s|%s)" % (re_number_1, re_number_2, re_number_3)).findall(inp)      
    
    except: 
        telephone = "no number"
    
    # get email
    print("What is the email address?")
    inp = input("You: ")
    
    try:
        email = re.findall('\S+@\S+', inp)
    
    except:
        email = "no email"
    
    # get office
    print("In which office is the new professor sitting?")
    inp = input("You: ")
    
    re_office_1 = r"[A-Z, a-z].\d{1}.\d{2}"
    re_office_2 = r"[A-Z, a-z].\d{3}"
    
    try:
        office = re.compile("(%s|%s)" % (re_office_1, re_office_2)).findall(inp)
    
    except:
        ofiice = "no office"
    
    # get research areas
    print("What are the research areas of the new professor?")
    inp = input("You: ")
    
    doc = nlp_research_area(inp)
    for ent in doc.ents:
        if ent.label_ == "RESEARCH_AREA":
            research_area = ent.text  
        else:
            research_area = "Not found"
    
    # get study
    print("In what study is the new professor teaching?")
    inp = input("You: ")
    doc = nlp_study(inp)
    for ent in doc.ents:
        if ent.label_ == "STUDY":
            study = ent.text  
        else:
            study = "Not found"
      
    # connect to database
    conn = sqlite3.connect("../PROF_INFO_DB.db")
    cur = conn.cursor()

    # insert new information into database
    cur.execute("INSERT INTO PROF_INFO_TABLE (prof_id, first_name,last_name, telephone, email, office) VALUES((SELECT MAX(prof_id) FROM PROF_INFO_TABLE)+1,?,?,?,?,?)", (first_name, last_name, telephone[0], email[0], office[0]))
    cur.execute("INSERT INTO PROF_RESEARCH_AREA_TABLE (prof_id, Name, research_area) VALUES((SELECT prof_id FROM PROF_INFO_TABLE WHERE last_name = ? ),?,?)", (last_name, last_name,research_area))
    cur.execute("INSERT INTO PROF_STUDY_TABLE (prof_id, Name, study) VALUES((SELECT prof_id FROM PROF_INFO_TABLE WHERE last_name = ? ),?,?)", (last_name, last_name,study))

    conn.commit()
    return 

### 4.5. Testen der Action Execution Funktion

In [18]:
def chat():
    log = []
    print("Start talking with me!(type quit to stop):")
    while True:
        inp = input("You: ")
        log.append(inp) #saves all input of user in list
        if inp.lower() == "quit":
            print("Goodbye :)")
            break
        
        # predict the intent
        results = model.predict([bag_of_words(inp, words)])[0] #output is just a probability for each label
        results_index = numpy.argmax(results) #index of greatest value
        intent = labels[results_index] #output is the most probable label
        print(intent)
        
        if intent == "extract_new_prof":
            # Action Execution
            action_execution_new_prof()
            # Response Generation
            response = "Thank you. I saved the new information in the database"
            print(response)
              

    return log # returns list of the inputs

In [19]:
chat()

Start talking with me!(type quit to stop):
You: there is a new professor
extract_new_prof
Great! What is the name of the new Professor?
You: peter blume
What is the phone number
You: +98 9999 999 9999
What is the email address?
You: peter.blume@mail.de
In which office is the new professor sitting?
You: t.333
What are the research areas of the new professor?
You: Machine Learning
In what study is the new professor teaching?
You: Win


OperationalError: no such table: PROF_INFO_TABLE